In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()
spark

25/11/23 11:48:02 WARN Utils: Your hostname, zurich-VivoBook-ASUSLaptop-X412FL-A412FL resolves to a loopback address: 127.0.1.1; using 192.168.1.32 instead (on interface wlo1)
25/11/23 11:48:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/23 11:48:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/23 11:48:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/23 11:48:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
# Example: Linear Regression with Spark MLlib
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Initialize Spark Session
spark = SparkSession.builder.appName('MLlib Example').getOrCreate()

# Load sample data
data = [(1, 5.0, 20.0), (2, 10.0, 25.0), (3, 15.0, 30.0), (4, 20.0, 35.0)]
columns = ['ID', 'Feature', 'Target']
df = spark.createDataFrame(data, columns)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=['Feature'], outputCol='Features')
df_transformed = assembler.transform(df)

# Train a linear regression model
lr = LinearRegression(featuresCol='Features', labelCol='Target')
model = lr.fit(df_transformed)

# Print model coefficients
print(f'Coefficients: {model.coefficients}')
print(f'Intercept: {model.intercept}')

25/11/23 11:48:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/11/23 11:48:57 WARN Instrumentation: [abab7750] regParam is zero, which might cause numerical instability and overfitting.
25/11/23 11:48:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/11/23 11:48:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Coefficients: [0.9999999999999992]
Intercept: 15.000000000000009


In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors

# Example dataset (sesuai materi)
data = [
    (1, Vectors.dense([2.0, 3.0]), 0),
    (2, Vectors.dense([1.0, 5.0]), 1),
    (3, Vectors.dense([2.5, 4.5]), 1),
    (4, Vectors.dense([3.0, 6.0]), 0)
]

columns = ['ID', 'Features', 'Label']
df = spark.createDataFrame(data, columns)

# Train logistic regression model
lr = LogisticRegression(featuresCol='Features', labelCol='Label')
model = lr.fit(df)

# Display coefficients and summary
print("Coefficients:", model.coefficients)
print("Intercept:", model.intercept)

Coefficients: [-12.262057946182315,4.087352272150037]
Intercept: 11.568912743674886


In [6]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors

# Example dataset (pakai Vectors.dense)
data = [
    (1, Vectors.dense([1.0, 1.0])),
    (2, Vectors.dense([5.0, 5.0])),
    (3, Vectors.dense([10.0, 10.0])),
    (4, Vectors.dense([15.0, 15.0]))
]

columns = ['ID', 'Features']
df = spark.createDataFrame(data, columns)

# Train KMeans clustering model
kmeans = KMeans(featuresCol='Features', k=2)
model = kmeans.fit(df)

# Show cluster centers
centers = model.clusterCenters()
print("Cluster Centers:", centers)

Cluster Centers: [array([5.33333333, 5.33333333]), array([15., 15.])]


In [9]:
#Load Dataset ke Spark & Persiapan Data
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("IrisClassification").getOrCreate()
df = spark.read.csv("file:///home/zurich/Downloads/Iris.csv",
                    header=True, inferSchema=True)

df.show(5)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



In [10]:
# Mengubah kolom label species menjadi angka
indexer = StringIndexer(inputCol="Species", outputCol="Label")
df_indexed = indexer.fit(df).transform(df)

assembler = VectorAssembler(
    inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"],
    outputCol="Features"
)

df_final = assembler.transform(df_indexed).select("Features", "Label")

df_final.show(5)


+-----------------+-----+
|         Features|Label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
+-----------------+-----+
only showing top 5 rows



In [14]:
#Membangun Model Klasifikasi
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="Features", labelCol="Label")
model = lr.fit(df_final)
print("Koefisien (Matrix):")
print(model.coefficientMatrix)

print("\nIntercept (Vector):")
print(model.interceptVector)

Koefisien (Matrix):
DenseMatrix([[-16.08836442,  64.64799121, -21.9737467 , -46.43057252],
             [  9.27679647, -28.9835485 ,   6.27216491,  14.07220185],
             [  6.81156795, -35.66444271,  15.70158179,  32.35837067]])

Intercept (Vector):
[33.325046405163036,4.656447471327328,-37.98149387649036]


In [16]:
#Evaluasi Model
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

pred = model.transform(df_final)

evaluator = MulticlassClassificationEvaluator(
    labelCol="Label",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = evaluator.evaluate(pred)
print("Akurasi Model:", accuracy)

Akurasi Model: 0.9866666666666667


In [19]:
#Hyperparameter Tuning (Cross-Validation)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.1, 0.5])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .build())
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)

cvModel = cv.fit(df_final)
bestModel = cvModel.bestModel

In [21]:
#Menampilkan model terbaik beserta tingkat Akurasi
print("Model Terbaik - regParam:", bestModel._java_obj.getRegParam())
print("Model Terbaik - elasticNetParam:", bestModel._java_obj.getElasticNetParam())

pred_best = bestModel.transform(df_final)
best_accuracy = evaluator.evaluate(pred_best)

print("Akurasi Terbaik Setelah Tuning:", best_accuracy)


Model Terbaik - regParam: 0.01
Model Terbaik - elasticNetParam: 0.5
Akurasi Terbaik Setelah Tuning: 0.9733333333333334
